In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import scipy.optimize as opt
import warnings
warnings.filterwarnings('ignore')

In [118]:
data = pd.read_excel("C:/Users/Niku/Documents/Semester8/DataFiles/datasetnorm1.xlsx")
data.shape

(1994, 121)

In [119]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [120]:
def costReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

In [121]:
data.insert(0, 'Ones', 1)

In [122]:
Normdata = data
Normdata = np.matrix(Normdata)
for i in range(1, Normdata.shape[1]):
    cur = Normdata[:, i]
    mx = max(cur)*1.0
    mn = min(cur)*1.0
    for j in range(0, len(cur)):
        cur[j] = (cur[j] - mn)/(mx - mn)
    Normdata[:, i] = cur

In [123]:
cols = Normdata.shape[1]
X = Normdata[:,1:cols-1]
y = Normdata[:,cols-1:cols]
theta = np.zeros(X.shape[1])

In [124]:
def gradientReg(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad

In [125]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.50 else 0 for x in probability]

In [126]:
y_pred = np.zeros(X_test.shape[0])
count = 0
prec = np.zeros(10)
recal = np.zeros(10)
f1 = np.zeros(10)
acc = np.zeros(10)
auc =  np.zeros(10)
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
for l in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
    learningRate = 1
    costReg(theta, X_train, y_train, learningRate)
    gradientReg(theta, X, y, learningRate)
    result = opt.fmin_tnc(func=costReg, x0=theta, fprime=gradientReg, args=(X_train, y_train, learningRate))
    theta_min = np.matrix(result[0])
    y_pred = predict(theta_min, X_test)
    prec[l] = precision_score(y_test, y_pred, average="weighted")
    recal[l] = recall_score(y_test, y_pred, average="weighted")
    f1[l] = f1_score(y_test, y_pred, average="weighted")
    acc[l] = accuracy_score(y_test, y_pred)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred) 
    auc[l] = metrics.auc(fpr, tpr)

In [127]:
np.mean(prec), np.mean(recal), np.mean(f1), np.mean(acc), np.mean(auc)

(0.92232414615873137,
 0.9212637913741224,
 0.9206934970534999,
 0.9212637913741224,
 0.91372340926929518)

In [128]:
precision_score(y_test, np.array(y_pred), average="weighted")

0.94182547642928782

In [129]:
y_test.shape, np.array(y_pred).shape

((997L, 1L), (997L,))